In [1]:
from datetime import datetime
import datetime as dt
import xarray as xr
import numpy as np
import glob
from dask.distributed import Client
import gc

from madmex.recipes import RECIPES
from datacube.api import GridWorkflow

from madmex.wrappers import gwf_query
from madmex.util.xarray import to_float, to_int
from django.contrib.gis.geos.geometry import GEOSGeometry
import json
from madmex.models import PredictObject



from operator import itemgetter

from affine import Affine
from rasterio.features import rasterize
from collections import OrderedDict
import fiona
import scipy
import pandas as pd


from madmex.overlay.conversions import rasterize_xarray
from madmex.util import chunk

from sklearn.cluster import KMeans

import geopandas as gpd
from madmex.util import join_dicts


2019-12-09 13:31:52,849 - datacube.drivers.driver_cache - driver_cache safe_load: Failed to resolve driver datacube.plugins.index::s3aio_index
2019-12-09 13:31:52,851 - datacube.drivers.driver_cache - driver_cache safe_load: Error was: UndefinedEnvironmentName("'extra' does not exist in evaluation environment.",)
distributed.client - ERROR - Restart timed out after 20.000000 seconds


In [2]:
scheduler_file = '/shared_volume/scheduler.json'

In [3]:
client = Client(scheduler_file=scheduler_file)

In [4]:
client

Client Scheduler: tcp://10.0.0.3:8786 Dashboard: http://10.0.0.3:8787/status,Cluster Workers: 8 Cores: 8 Memory: 416.00 GB


In [5]:
client.restart()

Client Scheduler: tcp://10.0.0.3:8786 Dashboard: http://10.0.0.3:8787/status,Cluster Workers: 8 Cores: 8 Memory: 416.00 GB


In [6]:
#region = 'Veracruz'
region = None
# shp_puntos_martinez
x = (-97.833251953125,-96.8395751953125)
y = (20.5, 19.6)
name_of_product = 's2_l2a_10m_scl_mexico'
begin = '2018-10-01'
end = '2019-12-31'
series_begin = '2018-10-04'
recipe_meta = RECIPES['s2_10m_scl_ndvi_mean_001']
product = recipe_meta['product']

gwf_kwargs = {'region': region, 'lat': y, 'long': x, 'product': name_of_product, 'begin': begin, 'end':end}

iterable = gwf_query(**gwf_kwargs)

In [7]:
dict_list = []
for prod in product:
    gwf_kwargs.update(product = prod)
    try:
        dict_list.append(gwf_query(**gwf_kwargs, view=False))
    # Exception is in case one of the product hasn't been registered in the datacube
    except Exception as e:
        pass
iterable = join_dicts(*dict_list, join='full').items()

In [8]:
segmentation = 'seg_mex_s2_10m_scl_10_18_09_19'
features_str = 'argmax_mean'
path = '/shared_volume/temp/segmentation_results/'
seg_intersec_path = '/shared_volume/tasks/2019/clusterization_Veracruz/intersections_merge.shp'

In [9]:
def prepare_clust_data(tile, segmentation_name, features_name, begin):
    sr_0 = xr.merge([GridWorkflow.load(x, dask_chunks={'x': 400, 'y': 400},
                                       measurements=['red','nir','pixel_qa']) for x in tile[1]])
#    sr_0 = GridWorkflow.load(tile[1],dask_chunks={'x': 400, 'y': 400}, measurements=['red','nir','pixel_qa'])
    # Load geoarray and feature collection
    geom = GEOSGeometry(json.dumps(tile[1][0].geobox.geographic_extent.json))
    crs = tile[1][0].geobox.crs
    sr_0.attrs['geobox'] = tile[1][0].geobox
    sr_0.attrs['crs'] = crs
    sr_0 = sr_0.apply(func=to_float, keep_attrs=True)
    # Keep clear pixels (2: Dark features, 4: Vegetation, 5: Not vegetated,
    # 6: Water, 7: Unclassified, 8: Cloud medium probability, 11: Snow/Ice)
    sr_1 = sr_0.where(sr_0.pixel_qa.isin([2,4,5,6,7,8,11]))
    # Compute ndvi
    sr_1['ndvi'] = ((sr_1.nir - sr_1.red) / (sr_1.nir + sr_1.red)) * 10000
    #will not use next bands
    sr_1 = sr_1.drop(['red', 'nir', 'pixel_qa'])
    
    #begin=np.datetime_as_string(sr_1['time'].values[0],unit='D')
    begin_time = datetime.strptime(begin, '%Y-%m-%d')
    
    ts_1 = sr_1.ndvi.resample(time='7D', 
                          loffset=begin_time - 
                          datetime.utcfromtimestamp(sr_1['time'][0].values.astype(datetime)*1e-9),
                          keep_attrs=True).nearest(tolerance=dt.timedelta(hours=84)).chunk({'time': -1}).interpolate_na('time')
    times=np.datetime_as_string(ts_1['time'],unit='D')
    ts_1_times_clean = ts_1.assign_coords({'time':times})
    
    if features_name is 'argmax_max':
        ts_1_argmax = ts_1_times_clean.argmax(axis=0)
        ts_1_argmax.name = 'ndvi_argmax'
        ts_1_max = ts_1_times_clean.max(axis=0)
        ts_1_max.name = 'ndvi_max'
    elif features_name is 'argmax_mean':
        ts_1_argmax = ts_1_times_clean.argmax(axis=0)
        ts_1_argmax.name = 'ndvi_argmax'
        ts_1_mean = ts_1_times_clean.mean(axis=0)
        ts_1_mean.name = 'ndvi_mean'
    else:
        ts_1_argmax = ts_1_times_clean.argmax(axis=0)
        ts_1_argmax.name = 'ndvi_argmax'
        ts_1_max = ts_1_times_clean.max(axis=0)
        ts_1_max.name = 'ndvi_max'
        ts_1_mean = ts_1_times_clean.mean(axis=0)
        ts_1_mean.name = 'ndvi_mean'
    
    aff = Affine(*list(sr_0.affine)[0:6])
    dimensions_dataset = list(sr_0.coords)
    list_dimensions = [x for x in dimensions_dataset if x != 'time']
    lambda_function = lambda l_netcdf,l_test: l_netcdf[0] if l_netcdf[0] in l_test else l_netcdf[1]
    xdim = lambda_function(list_dimensions,['x','longitude'])
    ydim = lambda_function(list_dimensions,['y','latitude'])
    out_shape_y = sr_0.sizes[ydim] 
    out_shape_x = sr_0.sizes[xdim]
    
    # Load geoarray and feature collection
#    geoarray = GridWorkflow.load(tile[1])
#    geom = GEOSGeometry(json.dumps(geoarray.geobox.geographic_extent.json))
    query_set = PredictObject.objects.filter(the_geom__contained=geom,
                                                 segmentation_information__name=segmentation_name)
    seg_id = query_set[0].id
    path = query_set[0].path
    with fiona.open(path) as src:
        fc = list(src)
        fc_subset=[(x['properties']['id'], x['geometry']) for x in fc]
    gdf = gpd.GeoDataFrame.from_features([feature for feature in fc], crs = crs.crs_str)
    gdf = gdf.rename(columns={'id':'features_id'})
    # Deallocate geoarray and feature collection
    
    fc_sorted = sorted(fc_subset, key=itemgetter(0))
    iterable_zip = zip([k[1] for k in fc_sorted], [k[0] for k in fc_sorted])
    
    fc_raster = rasterize(iterable_zip, transform=aff,
                          out_shape=(out_shape_y, out_shape_x),
                          dtype='float64', fill=np.nan)
    
    fc_raster = fc_raster.astype(int)
    fc_dataarray = xr.DataArray(fc_raster, dims=[ydim, xdim], name='features_id')    
    
    if features_name is 'argmax_max': 
        combined = xr.merge([fc_dataarray, ts_1_argmax, ts_1_max])
        
        combined = combined.stack(z=(xdim, ydim))
        combined=combined.reset_index('z')
        combined=combined.drop([xdim,ydim])
        combined = combined.where((np.isfinite(combined['features_id'])), drop=True)
        df = combined.to_dataframe()
        sr_0 = sr_1 = ts_1 = ts_1_times_clean = fc = fc_sorted = iterable_zip = fc_raster = fc_dataarray = combined = None
        gc.collect()
        df = df.sort_values(by='features_id')
        df1 = df.groupby('features_id').agg({'ndvi_argmax': lambda x: scipy.stats.mode(x)[0]}).reset_index()
        df2 = pd.merge(df1, df, on=['features_id', 'ndvi_argmax'], how='left')
        df3 = df2.groupby('features_id').agg({'ndvi_max': 'mean'}).reset_index()
        dff = pd.merge(df1, df3, on=['features_id'], how='left')
        df = df1 = df2 = df3 = None
        gc.collect()

    elif features_name is 'argmax_mean':
        combined = xr.merge([fc_dataarray, ts_1_argmax, ts_1_mean])
    
        combined = combined.stack(z=(xdim, ydim))
        combined=combined.reset_index('z')
        combined=combined.drop([xdim,ydim])
        combined = combined.where((np.isfinite(combined['features_id'])), drop=True)
        df = combined.to_dataframe()    
        sr_0 = sr_1 = ts_1 = ts_1_times_clean = fc = fc_sorted = iterable_zip = fc_raster = fc_dataarray = combined = None
        gc.collect()
        df = df.sort_values(by='features_id')
        df1 = df.groupby('features_id').agg({'ndvi_argmax': lambda x: scipy.stats.mode(x)[0]}).reset_index()
        df2 = pd.merge(df1, df, on=['features_id', 'ndvi_argmax'], how='left')
        df3 = df2.groupby('features_id').agg({'ndvi_mean': 'mean'}).reset_index()
        dff = pd.merge(df1, df3, on=['features_id'], how='left')
        df = df1 = df2 = df3 = None
        gc.collect()
    else:
        combined = xr.merge([fc_dataarray, ts_1_argmax, ts_1_max, ts_1_mean])
    
        combined = combined.stack(z=(xdim, ydim))
        combined=combined.reset_index('z')
        combined=combined.drop([xdim,ydim])
        combined = combined.where((np.isfinite(combined['features_id'])), drop=True)
        df = combined.to_dataframe()
        sr_0 = sr_1 = ts_1 = ts_1_times_clean = fc = fc_sorted = iterable_zip = fc_raster = fc_dataarray = combined = None
        gc.collect()
        df = df.sort_values(by='features_id')
        df1 = df.groupby('features_id').agg({'ndvi_argmax': lambda x: scipy.stats.mode(x)[0]}).reset_index()
        df2 = pd.merge(df1, df, on=['features_id', 'ndvi_argmax'], how='left')
        df3 = df2.groupby('features_id').agg({'ndvi_max': 'mean', 'ndvi_mean': 'mean'}).reset_index()
        dff = pd.merge(df1, df3, on=['features_id'], how='left')
        df = df1 = df2 = df3 = None
        gc.collect()
    
    gdf_df_join = gdf.merge(dff, on='features_id', how='left')
    dff = gdf = None
    gc.collect()

    return tile_name, gdf_df_join  


In [10]:
C = client.map(prepare_clust_data,
                       iterable,
                       pure=False,
                       **{'segmentation_name': segmentation,
                          'features_name': features_str,
                          'begin': series_begin
                          })

In [11]:
gdp_list_all = client.gather(C)

ValueError: All NaN slice encountered

In [13]:
client.restart()

Client Scheduler: tcp://10.0.0.3:8786 Dashboard: http://10.0.0.3:8787/status,Cluster Workers: 8 Cores: 8 Memory: 416.00 GB


In [ ]:
gdp_list_all = [x[1] for x in gdp_list_all]

In [11]:
gdf_df_join = pd.concat(gpd_list).set_index('features_id').reset_index().sort_index()

In [ ]:
gpd_list = None

In [ ]:
kmeans = KMeans(n_clusters=30)
if features_str is 'argmax_max':
    kmeans.fit(gdf_df_join[['ndvi_argmax','ndvi_max']].fillna(0).values)
elif features_str is 'argmax_mean':
    kmeans.fit(gdf_df_join[['ndvi_argmax','ndvi_mean']].fillna(0).values)
else:
    kmeans.fit(gdf_df_join[['ndvi_argmax','ndvi_max','ndvi_mean']].fillna(0).values)

In [ ]:
gdf_df_join = None

In [ ]:
# Get centers of the clusters
centers = kmeans.cluster_centers_

In [ ]:
def predict_clusters(gdf, kmeans, features_name, path, seg_intersec_path):
    try:
        tile_name = gdf[0]
        if features_name is 'argmax_max':
            name_file = 'ndvi_argmax_max_'+ str(tile_name[0]) + '_' + str(tile_name[1]) + '_05_12_2019_sen2_bis_t_40_s_0.5_c_0.7.shp'
            gdf_df_join = gdf[1]
            gdf = None
            gc.collect()
            preds = kmeans.predict(gdf_df_join[['ndvi_argmax','ndvi_max']].fillna(0).values)
            gdf_df_join['preds'] = preds
            name_file_ndvi = name_file.split('.shp')[0] + '_ndvi' + '.shp'
            gdf_df_join_ndvi = gdf_df_join.drop(['preds'],axis=1)
            gdf_df_join_ndvi.to_file(path + name_file_ndvi)
            name_file_cluster = name_file.split('.shp')[0] + '_cluster' + '.shp'
            gdf_df_cluster = gdf_df_join.drop(['ndvi_argmax','ndvi_max'],axis=1)
            gdf_df_cluster.to_file(path + name_file_cluster)            
            gdf_df_join = gdf_df_join_ndvi = None
            gc.collect()
        elif features_name is 'argmax_mean':
            name_file = 'ndvi_argmax_mean_'+ str(tile_name[0]) + '_' + str(tile_name[1]) + '_05_12_2019_sen2_bis_t_40_s_0.5_c_0.7.shp'
            gdf_df_join = gdf[1]
            gdf = None
            gc.collect()
            preds = kmeans.predict(gdf_df_join[['ndvi_argmax','ndvi_mean']].fillna(0).values)
            gdf_df_join['preds'] = preds
            name_file_ndvi = name_file.split('.shp')[0] + '_ndvi' + '.shp'
            gdf_df_join_ndvi = gdf_df_join.drop(['preds'],axis=1)
            gdf_df_join_ndvi.to_file(path + name_file_ndvi)
            name_file_cluster = name_file.split('.shp')[0] + '_cluster' + '.shp'
            gdf_df_cluster = gdf_df_join.drop(['ndvi_argmax','ndvi_mean'],axis=1)
            gdf_df_cluster.to_file(path + name_file_cluster)
            gdf_df_join = gdf_df_join_ndvi = None
            gc.collect()
        else:
            name_file = 'ndvi_argmax_max_mean'+ str(tile_name[0]) + '_' + str(tile_name[1]) + '_05_12_2019_sen2_bis_t_40_s_0.5_c_0.7.shp'
            gdf_df_join = gdf[1]
            gdf = None
            gc.collect()
            preds = kmeans.predict(gdf_df_join[['ndvi_argmax','ndvi_max','ndvi_mean']].fillna(0).values)
            gdf_df_join['preds'] = preds
            name_file_ndvi = name_file.split('.shp')[0] + '_ndvi' + '.shp'
            gdf_df_join_ndvi = gdf_df_join.drop(['preds'],axis=1)
            gdf_df_join_ndvi.to_file(path + name_file_ndvi)
            name_file_cluster = name_file.split('.shp')[0] + '_cluster' + '.shp'
            gdf_df_cluster = gdf_df_join.drop(['ndvi_argmax','ndvi_max','ndvi_mean'],axis=1)
            gdf_df_cluster.to_file(path + name_file_cluster)
            gdf_df_join = gdf_df_join_ndvi = None
            gc.collect()
        # Save shapefile with segments intersected with shp_puntos_martinez_reprojected.shp
        #gdf_df_cluster.assign(dctile=str(tile_name[0]) + '_' + str(tile_name[1]))
        gpds = gpd.read_file(seg_intersec_path)
        gpds = gpds.rename(columns={'id':'features_id'})
        gpds_cluster = gpds.merge(gdf_df_cluster, on=['features_id', 'geometry'], how='inner')
        name_file_gpds_cluster = name_file.split('.shp')[0] + '_cluster_segments' + '.shp'
        gpds_cluster.to_file(path + name_file_gpds_cluster)
        gpds_cluster = gdf_df_cluster = None
        return tile_name
    except Exception as e:
        print(e)
        return None

In [ ]:
C = client.map(predict_clusters,
                       gpd_list_all,
                       pure=False,
                       **{'kmeans': kmeans,
                          'features_name': features_str,
                          'path': path,
                          'seg_intersec_path': seg_intersec_path
                          })

In [ ]:
results = client.gather(C)

In [ ]:
client.restart()